In [17]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

ModuleNotFoundError: No module named 'sklearn'

In [4]:
train = pd.read_csv('path/to/file.csv',encoding="ISO-8859-1")
test = pd.read_csv('path/to/file.csv', encoding = "utf-8")

FileNotFoundError: File b'path/to/file.csv' does not exist

In [5]:
print('Total rows in test is {}'.format(len(test)))  
print('Total rows in train is {}'.format(len(train)))
cols_target = list(train)[2:]

NameError: name 'test' is not defined

In [6]:
data = train[cols_target]   


NameError: name 'train' is not defined

In [7]:
def clean_text(text):
    text = text.lower()
    text = text.strip(' ')
    return text


In [8]:
test['title'] = test['title'].map(lambda com : clean_text(com))
train['title'] = train['title'].map(lambda com : clean_text(com))


NameError: name 'test' is not defined

In [9]:
X = train.title

NameError: name 'train' is not defined

In [10]:
test_X = test.title

NameError: name 'test' is not defined

In [11]:
print(X.shape, test_X.shape)


NameError: name 'X' is not defined

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')


ModuleNotFoundError: No module named 'sklearn'

In [13]:
X_dtm = vect.fit_transform(X)

NameError: name 'vect' is not defined

In [14]:
test_X_dtm = vect.transform(test_X)

NameError: name 'vect' is not defined

In [16]:
# import and instantiate the Logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=12.0)

# create submission file
sample_submission = pd.read_csv('path/to/file.csv', encoding = "utf-8")
for label in cols_target:
    print('... Processing {}'.format(label))
    y = train[label]
    # train the model using X_dtm & y
    logreg.fit(X_dtm, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(test_X_dtm)[:,1]
    submission_binary[label] = test_y_prob
submission_binary.to_csv('submission_binary.csv',index=False)
submission_binary.describe()

ModuleNotFoundError: No module named 'sklearn'